In [1]:
import sys, os, _pickle as pickle
import tensorflow as tf
import numpy as np
from tensorflow.contrib import learn

In [2]:
data_dir = 'data'
ckpt_dir = 'checkpoint'
word_embd_dir = 'checkpoint/word_embd_100'
model_dir = 'checkpoint/model'

In [3]:
f = open('train_data.pkl', 'rb')
data, relation, e1, e2, e1_pos, e2_pos = pickle.load(f)

In [4]:
max_document_length = max([len(x.split(" ")) for x in data])

In [5]:
max_document_length

97

In [15]:
word_embd_dim = 100
pos_embd_dim = 25
dep_embd_dim = 25
vocab_size = 400000
pos_size = 10
relation_classes = 19
state_size = 100
max_sequence_length = 97
batch_size = 10

In [7]:
vocab_processor = learn.preprocessing.VocabularyProcessor.restore(os.path.join(data_dir, "word_vocab100"))

In [23]:
with tf.name_scope("input"):
    sequence_length = tf.placeholder(tf.int32, shape=[1])
    word_ids = tf.placeholder(tf.int32, shape=[max_sequence_length])
#     pos_tag = tf.placeholder(tf.int32, [None, 1], name="pos_tag")
#     relation = tf.placeholder(tf.int32, [None, relation_classes], name="relation")

In [24]:
with tf.name_scope("word_embedding"):
    W = tf.Variable(tf.constant(0.0, shape=[vocab_size, word_embd_dim]), name="W")
    embedding_placeholder = tf.placeholder(tf.float32,[vocab_size, word_embd_dim])
    embedding_init = W.assign(embedding_placeholder)
    embedded_word = tf.nn.embedding_lookup(W, word_ids)
    word_embedding_saver = tf.train.Saver({"word_embedding/W": W})

In [25]:
# with tf.name_scope("pos_embedding"):
#     W = tf.Variable(tf.random_uniform([pos_size, pos_embd_dim], name="W"))
#     embedded_pos = tf.nn.embedding_lookup(W, pos_tag)
#     pos_embedding_saver = tf.train.Saver({"pos_embedding/W": W})

In [26]:
hidden_state = tf.zeros([1, state_size], name='hidden_state')
cell_state = tf.zeros([1, state_size], name='cell_state')

In [27]:
hidden_states = []
cell_states = []

In [28]:
embedded_word

<tf.Tensor 'word_embedding_2/embedding_lookup:0' shape=(97, 100) dtype=float32>

In [13]:
import lstm
lstm.lstm_weights("word", word_embd_dim, state_size)

In [43]:
embd_word_series = tf.split(embedded_word, sequence_length, 0)

In [44]:
for i in range(max_sequence_length):
    if(sequence_length==i):
        l = i

In [45]:
embd_word_series

[<tf.Tensor 'split_1:0' shape=(?, ?) dtype=float32>]

In [46]:
embedded_word

<tf.Tensor 'word_embedding_2/embedding_lookup:0' shape=(97, 100) dtype=float32>

In [24]:
embd_word_series

[<tf.Tensor 'split:0' shape=(?, ?, ?) dtype=float32>]

In [30]:
for i in range(l):
    hidden_state, cell_state = lstm.lstm_cell("word", embd_word[i,:,:], hidden_state, cell_state)
    if i==0:
        hidden_states = hidden_state
    else:
        hidden_states = tf.concat([hidden_states, hidden_state], 0)
    l = l+1

TypeError: 'Tensor' object cannot be interpreted as an integer

In [41]:
# for i in range(2):
#     if i==0:
#         hidden_states = hidden_state
#     else:
#         hidden_states = tf.concat([hidden_states, hidden_state], 0)

In [42]:
hidden_states

<tf.Tensor 'concat_4:0' shape=(2, 100) dtype=float32>

In [27]:
states = tf.concat([hidden_state, cell_state], 1)

In [28]:
cell_state

<tf.Tensor 'word_1/add_8:0' shape=(?, 100) dtype=float32>

In [29]:
states

<tf.Tensor 'concat_1:0' shape=(?, 200) dtype=float32>

In [47]:
sess  =tf.Session()
sess.run(tf.global_variables_initializer())

In [48]:
# sess.run(embedding_init, feed_dict={embedding_placeholder: word_embd})
# word_embedding_saver.save(sess, word_embd_dir + '/word_embd')


In [49]:
latest_embd = tf.train.latest_checkpoint(word_embd_dir)
word_embedding_saver.restore(sess, latest_embd)

INFO:tensorflow:Restoring parameters from checkpoint/word_embd_100/word_embd


In [50]:
input_word = ["i","is"]
x = np.array(list(vocab_processor.transform(input_word)))
print(np.shape(x)[0])
# print(len(x))
len1 = np.array([np.shape(x)[0]])
print(len1)

2
[2]


In [56]:
np.shape(len1)

(1,)

In [58]:
t = sess.run(l, {word_ids:x, sequence_length:len1})
print(np.shape(t))
t

ValueError: Cannot feed value of shape (2, 1) for Tensor 'input_3/Placeholder_1:0', which has shape '(97,)'

In [20]:
io = tf.unstack(t,axis=0)

In [64]:
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import array_ops
from tensorflow.python.util import nest
from tensorflow.python.framework import ops
from tensorflow.python.framework import tensor_shape


In [47]:
if sequence_length is not None:
    sequence_length = math_ops.to_int32(sequence_length)

In [51]:
 sequence_length = array_ops.identity(  # Just to find it in the graph.
        sequence_length, name="sequence_length")

In [52]:
sequence_length

<tf.Tensor 'sequence_length:0' shape=(1,) dtype=int32>

In [67]:
flat_input = nest.flatten(embedded_word)

In [68]:
flat_input

[<tf.Tensor 'word_embedding_1/embedding_lookup:0' shape=(1, ?, 100) dtype=float32>]

In [69]:
def _transpose_batch_time(x):
  """Transpose the batch and time dimensions of a Tensor.
  Retains as much of the static shape information as possible.
  Args:
    x: A tensor of rank 2 or higher.
  Returns:
    x transposed along the first two dimensions.
  Raises:
    ValueError: if `x` is rank 1 or lower.
  """
  x_static_shape = x.get_shape()
  if x_static_shape.ndims is not None and x_static_shape.ndims < 2:
    raise ValueError(
        "Expected input tensor %s to have rank at least 2, but saw shape: %s" %
        (x, x_static_shape))
  x_rank = array_ops.rank(x)
  x_t = array_ops.transpose(
      x, array_ops.concat(
          ([1, 0], math_ops.range(2, x_rank)), axis=0))
  x_t.set_shape(
      tensor_shape.TensorShape([
          x_static_shape[1].value, x_static_shape[0].value
      ]).concatenate(x_static_shape[2:]))
  return x_t

In [70]:
flat_input = [ops.convert_to_tensor(input_) for input_ in flat_input]
flat_input = tuple(_transpose_batch_time(input_) for input_ in flat_input)

In [74]:
inputs = embedded_word

In [72]:
input_shape = tuple(array_ops.shape(input_) for input_ in flat_input)


In [75]:
inputs = nest.pack_sequence_as(structure=inputs, flat_sequence=flat_input)


In [76]:
inputs

<tf.Tensor 'transpose_2:0' shape=(?, 1, 100) dtype=float32>

In [77]:
  flat_input = nest.flatten(inputs)

In [82]:
flat_input

[<tf.Tensor 'transpose_2:0' shape=(?, 1, 100) dtype=float32>]

In [85]:
input_shape = array_ops.shape(flat_input)
# time_steps = input_shape[0]

In [90]:
input_shape[3]

<tf.Tensor 'strided_slice_6:0' shape=() dtype=int32>